In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
def gen_station_names(amount):
    """
    Creates names for stations given an amount of stations.
    """
    name_list = ['Root']
    for name in range(amount):
        name_list.append(chr(65 + name))
    return name_list

names = gen_station_names(5)
names

['Root', 'A', 'B', 'C', 'D', 'E']

In [3]:
def gen_locations(name_list):
    """
    Given a list of station names, generates random locations.
    """
    location_list = []
    max_xy = math.ceil(len(name_list) / 2)

    while len(location_list) < len(name_list):

        new_location = [np.random.randint(0, max_xy), np.random.randint(0, max_xy)]
        if new_location not in location_list:
            location_list.append(new_location)
    return location_list

locations = gen_locations(names)
print(len(names), len(locations))
locations

6 6


[[0, 0], [0, 1], [0, 2], [1, 0], [2, 2], [1, 1]]

In [17]:
location_list = locations
station = 2
[location for location in location_list if location_list[station][0] == location[0] or location_list[station][1] == location[1]]


In [4]:
def gen_connections(name_list, location_list):
    """
    Takes a list of names and corresponding locations and generates connections between them.
    """
    for station in range(name_list):
        possible_connections = [location for location in location_list if location_list[station][0] == location[0] or location_list[station][1] == location[1]]

In [5]:
def gen_connections(name_list, location_list):
    """
    Given a list of names and locations, generates random connections between them.
    """
    connection_list = []

    for name in name_list:

        temp_list = []
        updated_name_list = name_list.copy()
        updated_name_list.remove(name)
        connections = np.random.randint(1,5)

        for con in range(connections):
            connection = random.choice(updated_name_list)
            updated_name_list.remove(connection)
            temp_list.append(connection)

        connection_list.append(temp_list)

    return connection_list

connection_list = gen_connections(names)
connection_list

TypeError: gen_connections() missing 1 required positional argument: 'location_list'